In [47]:
# How extraction works ???
import os
import sys
import shutil
from pdf_processing_lib import PDFProcessor, ImageProcessor, TextExtractor, TSVJSON
os.chdir("/Users/aapoliakova/PycharmProjects/RAG_app_mozzilla")
import pathlib

In [51]:
pathlib.Path('/Users/aapoliakova/PycharmProjects/RAG_app_mozzilla/notebooks/app_storage/pdfs').exists()

True

In [56]:
def process_files():
    """Handle processing of uploaded PDFs through the complete pipeline"""
    try:
        # Get the storage directory path
        pdf_storage_dir = 'notebooks/app_storage/pdfs'
        output_base_dir = 'notebooks/processed'
        os.makedirs(output_base_dir, exist_ok=True)

        # Step 1: Initial PDF Processing
        print("Step 1: Initial PDF processing...")
        pdf_processor = PDFProcessor(pdf_storage_dir, output_base_dir)
        pdf_processor.process_directory()

        # Step 2: Image Processing with YOLO
        print("\nStep 2: Processing images with YOLO...")
        model_path = 'models/yolo.pt'  # You'll need to provide the path to your YOLO model
        image_processor = ImageProcessor(model_path)
        image_processor.process_directory(output_base_dir)

        # Step 3: Text Extraction
        print("\nStep 3: Extracting text...")
        text_extractor = TextExtractor(output_base_dir, pdf_storage_dir)
        text_extractor.process_directory_for_text_extraction()

        # Step 4: Final TSV to JSON conversion
        print("\nStep 4: Converting to final JSON format...")
        tsv_processor = TSVJSON(output_base_dir)
        final_json_path = os.path.join(output_base_dir, 'final_output.json')
        results = tsv_processor.process(final_json_path)

        print("\nProcessing completed successfully!")
        print(f"Results saved to: {output_base_dir}")
        return results

    except Exception as e:
            print(f"Error during processing: {str(e)}")
        
        
        


In [57]:
results = process_files()

Step 1: Initial PDF processing...


Processing PDFs: 100%|██████████| 2/2 [00:00<00:00, 1747.99it/s]


Total pages processed: 10
Total processing time: 0.36 seconds
Average time per page: 0.04 seconds/page

Step 2: Processing images with YOLO...

0: 480x352 1 GraphicZone, 4 GraphicZone-FigDescs, 1 GraphicZone-Head, 1 MainZone-Head, 1 NumberingZone, 1 PageTitleZone, 1 RunningTitleZone, 189.9ms
1: 480x352 11 MainZone-Ps, 1 NumberingZone, 189.9ms
2: 480x352 2 MainZone-Heads, 8 MainZone-Lists, 11 MainZone-Ps, 1 NumberingZone, 189.9ms
3: 480x352 5 MainZone-Ps, 1 NumberingZone, 189.9ms
4: 480x352 13 MainZone-Ps, 1 NumberingZone, 189.9ms
Speed: 1.6ms preprocess, 189.9ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 352)

0: 480x352 1 GraphicZone, 4 GraphicZone-FigDescs, 1 GraphicZone-Head, 1 MainZone-Head, 1 NumberingZone, 1 PageTitleZone, 1 RunningTitleZone, 159.2ms
1: 480x352 2 MainZone-Heads, 8 MainZone-Lists, 11 MainZone-Ps, 1 NumberingZone, 159.2ms
2: 480x352 11 MainZone-Ps, 1 NumberingZone, 159.2ms
3: 480x352 13 MainZone-Ps, 1 NumberingZone, 159.2ms
4: 480x352 5 MainZone-Ps

/opt/miniconda3/envs/pyqt_designer/lib/python3.10/site-packages/pdf_processing_lib/text_extractor.py:112: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  page_df = page_df[page_df['class_name'].str.contains(filter_strings, case=False, regex=True)]
/opt/miniconda3/envs/pyqt_designer/lib/python3.10/site-packages/pdf_processing_lib/text_extractor.py:124: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([bareme_df, page_df], axis=0, ignore_index=True)
/opt/miniconda3/envs/pyqt_designer/lib/python3.10/site-packages/pdf_processing_lib/text_extractor.py:112: UserWarning: This pattern is interpreted as a regular expression, and has match grou

Total combined_pageX.tsv files produced: 10
Total processing time: 1.73 seconds
Average processing time per file: 0.17 seconds
CVAT task XML file created: notebooks/processed/cvat/multi_document_task.xml

Step 4: Converting to final JSON format...
Starting complete document processing pipeline for: notebooks/processed
Step 1: Processing TSV files to JSON...
Found 10 TSV files to process in 2 directories


Processing directories: 100%|██████████| 2/2 [00:00<00:00, 186.99it/s]



Processed 2 final directories
Successfully processed 10/10 files
Total processing time: 0.01 seconds
Average time per file: 0.0012 seconds

Step 2: Merging all JSON files...


Merging JSON files: 100%|██████████| 2/2 [00:00<00:00, 3723.31it/s]


Combined JSON has been written to notebooks/processed/final_output.json
Total entries in merged file: 6

Processing completed!
Final output file: notebooks/processed/final_output.json
Total TSV files processed: 10
Total entries in final merged JSON: 6

Processing completed successfully!
Results saved to: notebooks/processed


In [58]:
results

{'final_output_file': 'notebooks/processed/final_output.json',
 'tsv_files_processed': 10,
 'total_processing_time': 0.011758804321289062,
 'avg_time_per_file': 0.0011758804321289062,
 'total_json_entries': 6}

In [59]:
import json
extraction_results_path  = results['final_output_file']
extraction_results_path
with open(extraction_results_path, 'r') as file:
    extractions_results = json.load(file)

In [60]:
print(extractions_results[1]['section'])

Test subject parsing 
- Anna has a green eyes.  
- Anna is 162 cm 
- Anna is 25 years old  
- Denis is 25 years old  
- Denis has pink eyes 
The overall conclusion is made based on target market.


In [62]:
from PyPDF2 import PdfReader


def extract_metadata(self, file_path):
    """Extract metadata from a PDF file using the PyPDF2 library.
    TO REPLACE WITH CARLOS FUNCTION"""
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        info = pdf_reader.metadata
        metadata = {
            'title': info.title if info.title else "Untitled",
            'author': info.author if info.author else "Unknown",
            'creation_date': info['/CreationDate'] if '/CreationDate' in info else "N/A",
            'subject': info.subject if info.subject else "N/A"
            
        }
    return metadata


In [ ]:
extract_metadata

In [42]:
extraction_results_path

'notebooks/processed/final_output.json'

In [ ]:
fake_metadata = {
    "file_path": 
}